# Instagram Telegram Bot Implementation Guide

This notebook provides a comprehensive guide to understanding, building, and deploying the Instagram Telegram Bot. The bot is designed to download and forward Instagram content to Telegram with robust error handling, rate limiting, and monitoring capabilities.

## Table of Contents
1. [Project Structure Analysis](#project-structure)
2. [Core Components Deep Dive](#core-components)
3. [Docker Configuration and Environment](#docker-config)
4. [Bot Implementation Analysis](#bot-implementation)
5. [Security and Authentication](#security)
6. [Rate Limiting and Resilience](#resilience)
7. [Building and Running the Bot](#build-and-run)

# Project Structure Analysis

The project follows a well-organized modular structure with clear separation of concerns. Here's an analysis of the main components:

## Directory Structure

```
src/
├── __init__.py
├── bot.py                 # Main bot entry point
├── core/                  # Core bot functionality
│   ├── monitoring/       # Logging and metrics
│   └── resilience/      # Rate limiting and error handling
├── services/             # Service implementations
└── utils/                # Utility functions
```

### Key Components

1. **Core Module**
   - Service management and dependency injection
   - Command routing and handler registration
   - Configuration management
   - Monitoring and metrics collection

2. **Services Module**
   - Instagram content download service
   - Telegram upload handlers
   - Session and database management
   - Rate limiting implementation

3. **Utils Module**
   - Helper functions and utilities
   - Size parsing and formatting
   - Progress tracking

This modular architecture allows for easy maintenance, testing, and future extensions of the bot's functionality.

# Core Components Deep Dive

Let's examine the core components that power the bot's functionality.

## Service Manager

The Service Manager is responsible for:
- Dependency injection and service lifecycle management
- Lazy initialization of services
- Cleanup and resource management

Here's a look at key interfaces and patterns:

In [ ]:
from typing import Dict, Type, Any

class ServiceManager:
    def __init__(self):
        self._services: Dict[Type, Any] = {}
        self._initialized = False
    
    async def get_service(self, service_type: Type) -> Any:
        if service_type not in self._services:
            self._services[service_type] = service_type()
            if hasattr(self._services[service_type], 'initialize'):
                await self._services[service_type].initialize()
        return self._services[service_type]
    
    async def cleanup(self):
        for service in self._services.values():
            if hasattr(service, 'cleanup'):
                await service.cleanup()

## Command Router

The Command Router handles:
- Registration of command handlers
- Message parsing and routing
- Error handling and logging

The router uses decorators to register command handlers and manages the flow of messages through the system.

In [ ]:
from typing import Callable, Dict, List
from functools import wraps

class CommandRouter:
    def __init__(self):
        self.handlers: Dict[str, List[Callable]] = {}
    
    def command(self, cmd_name: str):
        def decorator(func: Callable):
            @wraps(func)
            async def wrapper(*args, **kwargs):
                try:
                    return await func(*args, **kwargs)
                except Exception as e:
                    # Log error and provide user feedback
                    logger.exception(f"Error in command {cmd_name}")
                    raise
            
            self.handlers.setdefault(cmd_name, []).append(wrapper)
            return wrapper
        return decorator
    
    async def handle_message(self, message):
        handlers = self.handlers.get(message.command, [])
        for handler in handlers:
            await handler(message)

# Service Architecture

## Base Service Implementation
The bot uses a robust service architecture based on the `BaseService` abstract class:

```python
class BaseService(ABC):
    def __init__(self):
        self._initialized = False
        self._shutdown = False
        self._tasks: set[asyncio.Task] = set()
    
    @abstractmethod
    async def initialize(self) -> None:
        """Initialize the service."""
        pass
        
    @abstractmethod
    async def shutdown(self) -> None:
        """Clean up resources."""
        pass
```

## Service Initialization Order
Services are initialized in a specific order to handle dependencies:

1. Database Service
2. Session Storage
3. Telegram Services
4. Instagram Services
5. Upload Services

# Docker Configuration and Environment

The bot uses a multi-stage Docker build for optimized image size and security. Let's analyze the key components:

## Dockerfile Analysis

The Dockerfile uses a multi-stage build with:
1. Builder stage for compiling dependencies
2. Runtime stage for minimal production image
3. Security hardening and proper permissions

Key features:
- Python 3.12 slim base image
- Non-root user execution
- Volume mounts for persistence
- Health check implementation

## Docker Compose Configuration

The `docker-compose.yml` file defines the service configuration:

```yaml
services:
  bot:
    build: 
      context: .
      dockerfile: Dockerfile
    volumes:
      - ./downloads:/app/downloads
      - ./data:/app/data:rw
      - ./sessions:/app/downloads/sessions:rw
      - ./uploads:/app/uploads
      - ./temp:/app/temp
      - ./telegram:/app/telegram
      - ./gallery-dl-cookies.txt:/app/gallery-dl-cookies.txt:ro
    environment:
      - PYTHONUNBUFFERED=1
      - MAX_CONCURRENT_DOWNLOADS=3
      - MAX_CONCURRENT_UPLOADS=3
    deploy:
      resources:
        limits:
          memory: 1G
          cpus: '0.75'
```

Key features:
1. Volume mounts for persistent data
2. Resource limits for CPU and memory
3. Environment variable configuration
4. Health check integration
5. Logging configuration

# Bot Implementation Analysis

Let's analyze how the bot handles messages and processes Instagram content. The main entry point is `bot.py`, which sets up the application and handles the event loop.

## Initialization Process

1. Load environment variables
2. Configure logging
3. Set up directory structure
4. Initialize services
5. Start command handling

Here's the core initialization code:

In [ ]:
async def initialize_bot(config: BotConfig) -> Optional[EnhancedTelegramBot]:
    """Initialize the bot with configuration."""
    try:
        bot = EnhancedTelegramBot(config)
        await bot.initialize()
        return bot
    except Exception as e:
        logger.exception("Failed to initialize bot", error=str(e))
        return None

async def main() -> int:
    try:
        # Load environment and configuration
        load_environment()
        config = load_configuration()
        
        # Set up logging and directories
        setup_structured_logging(config.logging)
        setup_directories(config)
        
        # Initialize and run bot
        bot = await initialize_bot(config)
        if bot is None:
            return 1
        
        await run_bot(bot)
        return 0
    except Exception as e:
        logger.exception("Fatal error in main loop")
        return 1

# Security and Authentication

The bot implements multiple layers of security:

1. **Telegram Authentication**
   - Bot token validation
   - Admin user verification
   - Session persistence

2. **Instagram Authentication**
   - Cookie-based authentication
   - Secure cookie storage
   - Session renewal handling

## Cookie Management

The bot uses Netscape-format cookies for Instagram authentication. Here's an example of the cookie file structure:

In [ ]:
# Example cookie file content
cookies = """# Netscape HTTP Cookie File
# https://curl.haxx.se/rfc/cookie_spec.html
# This is a generated file! Do not edit.

.instagram.com	TRUE	/	TRUE	1792500164	datr	xAnIaNrfKV09uRK93nH83dV9
.instagram.com	TRUE	/	TRUE	1789476202	ig_did	B98E2B26-0BF5-4614-88C2-8685D4C4A916
.instagram.com	TRUE	/	TRUE	1792500166	mid	aMgJxAAEAAG_eaUHYPwm-VDA1OZv
.instagram.com	TRUE	/	TRUE	1767452355	ds_user_id	72439960661
.instagram.com	TRUE	/	TRUE	1791212338	sessionid	72439960661%3AGB82GHH46aJMxF%3A26"""

# The cookies are loaded securely and validated before use
# Important fields: sessionid, ds_user_id, csrftoken

# Rate Limiting and Resilience

The bot implements sophisticated rate limiting and resilience features:

## Rate Limiting

1. **API Rate Limits**
   - Instagram API limits
   - Telegram message limits
   - Concurrent operation limits

2. **Circuit Breaker Pattern**
   - Prevents API abuse
   - Handles service outages
   - Implements backoff strategies

Here's the core rate limiting implementation:

# Enhanced Rate Limiting and Resilience

The bot implements sophisticated rate limiting and resilience features:

## Rate Limiting Implementation

1. Instagram-specific rate limiting:
- Request rate tracking
- Session rotation
- Conservative mode
- Burst protection

2. Smart backoff strategies:
- Exponential backoff
- Jitter
- Error-based delays

Here's the core rate limiting code:

In [ ]:
class InstagramRateLimiter:
    """Handles rate limiting for Instagram API requests."""
    
    def __init__(self, config_path: Optional[Path] = None):
        self.config = RateLimitConfig(
            config_path or Path('config/rate_limiting.conf')
        )
        self.last_request_time = 0.0
        self.request_count = 0
        self.error_count = 0
        self.session_start_time = datetime.now()
        self.in_conservative_mode = False
        
    async def wait_for_request(self, request_type: str = 'normal') -> None:
        """Smart wait before making a request."""
        current_time = time.time()
        time_since_last = current_time - self.last_request_time
        
        # Calculate base delay
        base_delay = max(0, self.config.MIN_REQUEST_INTERVAL - time_since_last)
        
        # Add backoff in conservative mode
        if self.in_conservative_mode:
            base_delay *= 2
            
        # Add jitter and wait
        delay = self._add_jitter(base_delay)
        if delay > 0:
            await asyncio.sleep(delay)
            
        self.last_request_time = time.time()

## Circuit Breaker Pattern

The bot implements a circuit breaker pattern to prevent cascading failures:

1. Failure tracking
2. Automatic circuit opening
3. Recovery with timeout
4. Service protection

Here's the circuit breaker implementation:

In [ ]:
class CircuitBreaker:
    """Circuit breaker to prevent cascade failures."""
    
    def __init__(self, failure_threshold: int = 5, reset_timeout: int = 60):
        self.failures = 0
        self.last_failure_time = 0
        self.threshold = failure_threshold
        self.reset_timeout = reset_timeout
        self._is_open = False

    def is_open(self) -> bool:
        """Check if circuit breaker is open."""
        if not self._is_open:
            return False
            
        # Check if enough time has passed to try again
        if time.time() - self.last_failure_time >= self.reset_timeout:
            self._is_open = False
            self.failures = 0
            return False
            
        return True

    def record_failure(self) -> None:
        """Record a failure and potentially open the circuit."""
        self.failures += 1
        self.last_failure_time = time.time()
        
        if self.failures >= self.threshold:
            self._is_open = True

## Smart Backoff Implementation

The bot uses smart backoff strategies to handle various error conditions:

1. Exponential backoff with jitter
2. Error-specific delays
3. Conservative mode triggers
4. Session rotation

Here's the smart backoff implementation:

In [ ]:
class SmartBackoff:
    def __init__(self):
        self.initial_delay = 10
        self.max_delay = 1800  # 30 minutes
        self.multiplier = 2
        self.jitter = 0.1
        
    def calculate_delay(self, attempts: int) -> float:
        """Calculate delay with exponential backoff and jitter."""
        delay = min(
            self.initial_delay * (self.multiplier ** attempts),
            self.max_delay
        )
        jitter_amount = delay * self.jitter
        return delay + random.uniform(-jitter_amount, jitter_amount)
        
    def handle_error(self, error: Exception) -> float:
        """Handle error and return appropriate backoff time."""
        error_str = str(error).lower()
        
        if "rate limit" in error_str:
            return self.calculate_delay(3)  # Aggressive backoff
        elif "blocked" in error_str:
            return self.max_delay  # Maximum backoff
        else:
            return self.calculate_delay(1)  # Normal backoff

In [ ]:
from datetime import datetime, timedelta
from typing import Dict, Optional

class RateLimiter:
    def __init__(self, requests_per_window: int, window_seconds: int):
        self.requests_per_window = requests_per_window
        self.window_seconds = window_seconds
        self.requests: Dict[datetime, int] = {}
    
    async def acquire(self) -> bool:
        now = datetime.now()
        window_start = now - timedelta(seconds=self.window_seconds)
        
        # Clean old entries
        self.requests = {
            ts: count for ts, count in self.requests.items()
            if ts > window_start
        }
        
        # Check current request count
        current_requests = sum(self.requests.values())
        if current_requests >= self.requests_per_window:
            return False
        
        # Record new request
        self.requests[now] = self.requests.get(now, 0) + 1
        return True

# Building and Running the Bot

This section covers the complete process of building and running the Instagram Telegram Bot.

## Prerequisites

1. Docker and Docker Compose installed
2. Telegram Bot token (from @BotFather)
3. Instagram account cookies
4. Python 3.12+ (for local development)

## Setup Process

1. Clone the repository
2. Create `.env` file from example.env
3. Configure Instagram cookies
4. Run setup script
5. Initialize the bot

Here are the steps in detail:

In [ ]:
# Clone the repository
git clone https://github.com/username/instagram-telegram-bot
cd instagram-telegram-bot

# Create and configure environment file
cp example.env .env
nano .env  # Edit with your Telegram credentials

# Run setup script
chmod +x setup.sh
./setup.sh

# Start the bot
docker compose up -d

# Check logs
docker compose logs -f

## Health Monitoring

The bot includes a comprehensive health monitoring system via `healthcheck.py`. It monitors:

1. System resources (CPU, memory, disk)
2. Database connectivity
3. Telegram API status
4. Service health

The health check runs every 30 seconds and provides detailed metrics.

In [ ]:
async def check_health():
    health_check = HealthCheck(Path(__file__).parent)
    results = await health_check.check_components()
    
    print(json.dumps(results, indent=2))
    return results['overall_status'] == 'healthy'

# Example health check output
{
    "timestamp": "2025-10-05T12:34:56Z",
    "system": {
        "cpu": {"percent": 25.3, "status": "healthy"},
        "memory": {"percent": 45.2, "status": "healthy"},
        "disk": {
            "data": {"percent": 65.0, "status": "healthy"}
        }
    },
    "database": {"status": "healthy", "latency": 0.023},
    "telegram_api": {"status": "healthy", "latency": 0.156},
    "overall_status": "healthy"
}

## Bot Usage

Once the bot is running, you can interact with it using these commands:

1. `/start` - Initialize bot and get help
2. `/login` - Authenticate with Telegram
3. `/session` - Upload Instagram cookies
4. `/status` - Check bot status
5. `/help` - View all commands

The bot will process Instagram URLs sent to it and forward the content to the configured Telegram chat.

## Maintenance

Regular maintenance tasks:
1. Monitor logs for errors
2. Update Instagram cookies when needed
3. Check disk space usage
4. Monitor rate limiting statistics
5. Update bot token if needed

Use the provided monitoring tools to ensure smooth operation of the bot.